In [1]:
import pandas as pd
import urllib

import numpy as np

import json

from tqdm.autonotebook import tqdm

#%matplotlib inline

tqdm.pandas()

import dask.dataframe as dd

from dask.multiprocessing import get
from dask.diagnostics import ProgressBar

from datetime import datetime
import matplotlib.pyplot as plt

from IPython.display import display


<ipython-input-1-5cfb607d7a2e>:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm
Matplotlib is building the font cache; this may take a moment.


In [2]:
import urllib3

In [3]:
http = urllib3.PoolManager()

In [4]:
from config_batch import * 

# Functions

In [5]:
ws_hostname = "127.0.1.1"
ws_hostname = "10.1.0.45"

# ws_hostname = "192.168.1.3"


In [6]:
def call_ws(addr_data, check_result=True, structured_osm=False): #lg = "en,fr,nl"
    t = datetime.now()
    
    params = urllib.parse.urlencode({"street": addr_data[street_field],
                                     "housenumber": addr_data[housenbr_field],
                                     "city": addr_data[city_field],
                                     "postcode": addr_data[postcode_field],
                                     "country": addr_data[country_field],
                                     "check_result" : "yes" if check_result else "no",
                                     "struct_osm" : "yes" if structured_osm else "no"
                                    })
    url = f"http://{ws_hostname}:5000/search/?{params}"
    
    try:
        with urllib.request.urlopen(url) as response:
            res = response.read()
            res = json.loads(res)
#             print(res)
            res["time"] = datetime.now() - t
            return res
    except Exception as e:
        return str(e)
    

In [7]:
def call_ws_batch(addr_data, mode="geo", with_reject=False, check_result=True, structured_osm=False): #lg = "en,fr,nl"
#     print(addr_data)
#     print(addr_data.shape)
#     print()
    file_data = addr_data.rename(columns = {
        street_field : "street",
        housenbr_field: "housenumber",
        postcode_field: "postcode",
        city_field: "city",
        country_field: "country",
        addr_key_field : "addr_key"
    }).to_csv(index=False)
    
    r = http.request(
    'POST',
    f'http://{ws_hostname}:5000/batch',
    fields= { 
        'media': ('addresses.csv', file_data),
        'mode': mode,
        "with_rejected" : "yes" if with_reject else "no",
        "check_result" : "yes" if check_result else "no",
        "struct_osm" : "yes" if structured_osm else "no"
    })
    
    try:
        res = pd.DataFrame(json.loads(r.data.decode('utf-8')))
    except ValueError:
        print("Cannot decode result:")
        print(json.loads(r.data.decode('utf-8')))
        return 
#     display(res)
    return res

In [8]:
def expand_json(addresses):
    addresses["status"]= addresses.json.apply(lambda d: "error" if "error" in d else "match" if "match" in d else "rejected")
    addresses["time"]  = addresses.json.apply(lambda d: d["time"])

    addresses["timing"]  = addresses.json.apply(lambda d: d["timing"] if "timing" in d else {})

    addresses["method"]= addresses.json.apply(lambda d: d["match"][0]["method"] if len(d)>0 and "match" in d else "none")
    
    for field in ["street", "number", "postcode", "city"]:
        addresses[field]= addresses.json.apply(lambda d: d["match"][0]["addr_out_"+field] if len(d)>0 and "match" in d else "")
    return 

# Calls

## Single address calls

In [9]:
call_ws({street_field:   "Av. Fonsny", 
         housenbr_field: "20",
         city_field:     "Saint-Gilles",
         postcode_field: "1060",
         country_field:  "Belgium"}, check_result=False, structured_osm=False)

{'match': [{'SIM_street_which': '',
   'addr_out_city': 'Saint-Gilles - Sint-Gillis',
   'addr_out_country': 'België / Belgique / Belgien',
   'addr_out_number': '20',
   'addr_out_postcode': '1060',
   'addr_out_street': 'Avenue Fonsny - Fonsnylaan',
   'display_name': 'DAE (Smals), 20, Avenue Fonsny - Fonsnylaan, Saint-Gilles - Sint-Gillis, Brussel-Hoofdstad - Bruxelles-Capitale, Région de Bruxelles-Capitale - Brussels Hoofdstedelijk Gewest, 1060, België / Belgique / Belgien',
   'extra_house_nbr': '20',
   'lat': '50.8358216',
   'lon': '4.3386884',
   'method': 'orig',
   'osm_addr_in': 'Av. Fonsny, 20, 1060 Saint-Gilles, Belgium',
   'place_id': 320313,
   'place_rank': 30}],
 'rejected': [{'SIM_street': '',
   'SIM_zip': '',
   'addr_out_city': 'Saint-Gilles - Sint-Gillis',
   'addr_out_country': 'België / Belgique / Belgien',
   'addr_out_number': '20',
   'addr_out_postcode': '1060',
   'addr_out_street': 'Avenue Fonsny - Fonsnylaan',
   'display_name': '20, Avenue Fonsny - Fon

In [160]:
call_ws({street_field:   "Avenue Louise", 
         housenbr_field: "375",
         city_field:     "Ixelles",
         postcode_field: "1050",
         country_field:  "Belgium"}, check_result=True, structured_osm=True)

{'match': [{'SIM_city': 1.0,
   'SIM_house_nbr': 0.0,
   'SIM_street': 1.0,
   'SIM_street_which': 'addr_out_street',
   'SIM_zip': 1.0,
   'addr_out_city': 'Ixelles - Elsene',
   'addr_out_country': 'België / Belgique / Belgien',
   'addr_out_number': '',
   'addr_out_postcode': '1050',
   'addr_out_street': 'Avenue Louis Lepoutre - Louis Lepoutrelaan',
   'display_name': 'Avenue Louis Lepoutre - Louis Lepoutrelaan, Ixelles - Elsene, Brussel-Hoofdstad - Bruxelles-Capitale, Région de Bruxelles-Capitale - Brussels Hoofdstedelijk Gewest, 1050, België / Belgique / Belgien',
   'extra_house_nbr': '375',
   'lat': '50.8196702',
   'lon': '4.3586694',
   'method': 'libpostal+regex[lpost]+photon',
   'osm_addr_in': 'Avenue Louis Lepoutre - Louis Lepoutrelaan, 375, 1050 Ixelles - Elsene, België - Belgique - Belgien',
   'place_id': 521112,
   'place_rank': 26}],
 'rejected': [],
 'time': datetime.timedelta(seconds=2, microseconds=789996)}

## Batch calls (row by row)

In [126]:
addresses = get_addresses("address.csv.gz")
addresses = addresses.sample(100).copy()

### Simple way

In [74]:
addresses["json"] = addresses.progress_apply(call_ws, check_result=True, structured_osm=False, axis=1)

  0%|          | 0/100 [00:00<?, ?it/s]

### Using Dask

In [17]:
dd_addresses = dd.from_pandas(addresses, npartitions=4)

dask_task = dd_addresses.apply(call_ws, meta=('x', 'str'), axis=1)

with ProgressBar(): 
    addresses["json"] = dask_task.compute()

[########################################] | 100% Completed | 59.1s


In [26]:
expand_json(addresses)

In [27]:
addresses

,EntityNumber,CountryFR,Zipcode,MunicipalityFR,StreetFR,HouseNumber,json,status,time,timing,method,street,number,postcode,city
1321156,0836.380.520,Belgique,7911,Frasnes-lez-Anvaing,Rue Goderneau(MB),14,"{'match': [{'SIM_street_which': '', 'addr_out_...",match,0 days 00:00:00.960761,{},regex[init],Rue Goderneau,,7906,Ath
2660061,2.231.595.460,Belgique,9981,Sint-Laureins,Vlamingstraat(STM),92,"{'match': [{'SIM_street_which': '', 'addr_out_...",match,0 days 00:00:01.143796,{},regex[init],Vlamingstraat,,9981,Sint-Laureins
164680,0431.356.723,Belgique,1050,Ixelles,Rue de l'Amazone,21,"{'match': [{'SIM_street_which': '', 'addr_out_...",match,0 days 00:00:00.724923,{},orig,Rue de l'Amazone - Amazonestraat,21,1050,Ixelles - Elsene
1295595,0832.380.556,Belgique,4000,Liège,Rue Henri-Maus,29,"{'match': [{'SIM_street_which': '', 'addr_out_...",match,0 days 00:00:00.584219,{},orig,Rue Henri Maus,,4000,Liège
2293307,2.168.200.715,Belgique,8300,Knokke-Heist,Zeedijk-Knokke,641,"{'match': [{'SIM_street_which': '', 'addr_out_...",match,0 days 00:00:00.548851,{},orig,Zeedijk Knokke,,8300,Knokke-Heist
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2751112,2.244.614.642,Belgique,9240,Zele,Smeiersberg,5,"{'match': [{'SIM_street_which': '', 'addr_out_...",match,0 days 00:00:00.544249,{},orig,Smeiersberg,,9240,Zele
2810314,2.252.606.947,Belgique,1080,Molenbeek-Saint-Jean,Rue Henri De Saegher,27,"{'match': [{'SIM_street_which': '', 'addr_out_...",match,0 days 00:00:00.968186,{},orig,Rue Henri De Saegher - Henri De Saegherstraat,27,1080,Molenbeek-Saint-Jean - Sint-Jans-Molenbeek
732484,0650.498.331,Belgique,4700,Eupen,Stockbergerweg,5,"{'match': [{'SIM_street_which': '', 'addr_out_...",match,0 days 00:00:00.987093,{},orig,Stockbergerweg,,4700,Eupen
1799145,2.037.557.947,Belgique,1180,Uccle,Parvis Saint-Pierre,119,"{'match': [{'SIM_street_which': '', 'addr_out_...",match,0 days 00:00:00.517703,{},orig,Parvis Saint-Pierre - Sint-Pietersvoorplein,,1180,Uccle - Ukkel


## Batch calls (batch WS)

### Single block

In [53]:
# Only geocoding
call_ws_batch(addresses, mode="geo", check_result=True)

,addr_key,lat,lon,place_rank,method
0,0308.251.647,50.467047,4.207648,26.0,orig
1,0408.704.352,50.894373,4.379952,30.0,orig
2,0413.358.372,51.220346,4.413114,30.0,orig
3,0431.356.723,50.824780,4.359237,30.0,orig
4,0439.768.306,50.190684,4.535742,26.0,orig
...,...,...,...,...,...
93,0453.040.676,50.270696,5.521461,26.0,nonum
94,0535.902.531,50.292862,5.031069,30.0,libpostal+regex[lpost]+photon
95,2.202.472.003,50.878578,4.409547,26.0,libpostal+regex[lpost]+photon
96,2.237.979.050,50.590356,5.747109,26.0,libpostal+regex[lpost]+photon


In [62]:
# Geocode + address
call_ws_batch(addresses, mode="short") 

,addr_key,lat,lon,place_rank,method,addr_out_street,addr_out_number,extra_house_nbr,addr_out_postcode,addr_out_city,addr_out_country
0,2.164.397.226,50.712641,4.530106,26.0,orig,Rue Robert Boisacq,None,9 A,1330,Rixensart,België / Belgique / Belgien
1,2.279.599.374,50.859702,2.828440,26.0,orig,Stijn Streuvelswijk,None,18,8908,Ieper,België / Belgique / Belgien
2,0669.833.203,51.209175,4.470072,30.0,orig,Boterlaarbaan,239,239,2100,Deurne,België / Belgique / Belgien
3,2.243.108.073,50.658359,5.483832,26.0,orig,Rue de l'Aéropostale,None,8,4460,Grâce-Hollogne,België / Belgique / Belgien
4,2.045.477.107,51.100407,3.167726,26.0,orig,Koningin Astridstraat,None,109,8210,Zedelgem,België / Belgique / Belgien
...,...,...,...,...,...,...,...,...,...,...,...
92,0857.952.627,50.834163,3.231044,26.0,regex[init],Heulsestraat,None,3,8501,Kortrijk,België / Belgique / Belgien
93,2.249.583.121,51.194730,4.407469,30.0,orig,Desguinlei,88-90,90,2018,Antwerpen,België / Belgique / Belgien
94,0834.315.707,51.204519,4.398727,30.0,orig,Diercxsensstraat,39,39,2018,Antwerpen,België / Belgique / Belgien
95,0857.635.002,50.843955,4.264415,26.0,orig,Ninoofsesteenweg,None,227,1700,Dilbeek,België / Belgique / Belgien


In [63]:
# Geocode + address, with rejected addresses
call_ws_batch(addresses, mode="long", with_reject=True) 

,addr_key,country,postcode,city,street,housenumber,osm_addr_in,place_id,lat,lon,...,addr_out_street,addr_out_city,addr_out_number,addr_out_country,addr_out_postcode,addr_out_other,retry_on_26,method,extra_house_nbr,rejected
0,2.164.397.226,Belgique,1330,Rixensart,Rue Robert Boisacq,9 A,"Rue Robert Boisacq, 9 A, 1330 Rixensart, Belgique",564275.0,50.712641,4.530106,...,Rue Robert Boisacq,Rixensart,None,België / Belgique / Belgien,1330,Fond du Patch,None,orig,9 A,None
1,2.279.599.374,Belgique,8908,Ieper,Streuvelswijk,18,"Streuvelswijk, 18, 8908 Ieper, Belgique",785053.0,50.859702,2.828440,...,Stijn Streuvelswijk,Ieper,None,België / Belgique / Belgien,8908,,None,orig,18,None
2,0669.833.203,Belgique,2100,Antwerpen,Boterlaarbaan,239,"Boterlaarbaan, 239, 2100 Antwerpen, Belgique",2010239.0,51.209175,4.470072,...,Boterlaarbaan,Deurne,239,België / Belgique / Belgien,2100,Eksterlaar,None,orig,239,None
3,2.243.108.073,Belgique,4460,Grâce-Hollogne,Rue de l'Aéropostale,8,"Rue de l'Aéropostale, 8, 4460 Grâce-Hollogne, ...",1445516.0,50.658359,5.483832,...,Rue de l'Aéropostale,Grâce-Hollogne,None,België / Belgique / Belgien,4460,Liège logistics,None,orig,8,None
4,2.045.477.107,Belgique,8210,Zedelgem,Koningin Astridstraat,109,"Koningin Astridstraat, 109, 8210 Zedelgem, Bel...",1763661.0,51.100407,3.167726,...,Koningin Astridstraat,Zedelgem,None,België / Belgique / Belgien,8210,,None,orig,109,"[{'index': 5.0, 'osm_addr_in': 'Koningin Astri..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,0857.952.627,Belgique,8501,Kortrijk,Heulsestraat(Bis),3,"Heulsestraat, 3, 8501 Kortrijk, Belgique",2738948.0,50.834163,3.231044,...,Heulsestraat,Kortrijk,None,België / Belgique / Belgien,8501,,None,regex[init],3,None
93,2.249.583.121,Belgique,2018,Antwerpen,Desguinlei,90,"Desguinlei, 90, 2018 Antwerpen, Belgique",2323301.0,51.194730,4.407469,...,Desguinlei,Antwerpen,88-90,België / Belgique / Belgien,2018,Markgrave,None,orig,90,None
94,0834.315.707,Belgique,2018,Antwerpen,Diercxsensstraat,39,"Diercxsensstraat, 39, 2018 Antwerpen, Belgique",2338350.0,51.204519,4.398727,...,Diercxsensstraat,Antwerpen,39,België / Belgique / Belgien,2018,Brederode,None,orig,39,None
95,0857.635.002,Belgique,1700,Dilbeek,Ninoofsesteenweg,227,"Ninoofsesteenweg, 227, 1700 Dilbeek, Belgique",943444.0,50.843955,4.264415,...,Ninoofsesteenweg,Dilbeek,None,België / Belgique / Belgien,1700,,None,orig,227,"[{'index': 120.0, 'osm_addr_in': 'Ninoofsestee..."


### Batch blocs

In [ ]:
def call_ws_batch_chunks(addr_data, mode="geo", with_reject=False, check_result=True, structured_osm=False, chunk_size=100): 
    ## TODO : find a better way with dask? It seems that map_partitions does not support function returning dataframes. 

    chunks = np.array_split(addr_data, addr_data.shape[0]//chunk_size)

    res= [call_ws_batch(chunk, mode=mode, 
                        check_result=check_result, 
                        structured_osm=structured_osm) for chunk in tqdm(chunks)]
    df_res = pd.concat(res, sort=False)
    return df_res

In [ ]:
df_res = call_ws_batch_chunks(addresses, chunk_size=10)
df_res

In [24]:
df_res.method.value_counts()

orig                             73
regex[init]                      19
libpostal+regex[lpost]+photon     3
nonum                             2
nostreet                          2
Name: method, dtype: int64

## Comparing options

In [127]:
addresses = get_addresses("address.csv.gz")
# addresses = addresses[addresses[country_field] == "Belgique"]
addresses = addresses.sample(10000).copy()

In [115]:
results = {}
it_per_seconds=pd.DataFrame()

for check_label in ["check", "nocheck"]:
    for struct_label in ["struct", "unstruct" ]:
        print(check_label, struct_label)
        start=datetime.now()
        
        results[(check_label, struct_label)] = call_ws_batch_chunks(addresses, 
                                                                    mode="short", 
                                                                    check_result   =  check_label == "check", 
                                                                    structured_osm =  struct_label == "struct")
        
        it_per_seconds.loc[check_label, struct_label] = addresses.shape[0] / (datetime.now()-start).total_seconds()
print("Iterations per seconds:")
it_per_seconds

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

Iterations per seconds:


,struct,unstruct
check,12.142767,11.979803
nocheck,16.449487,13.823085


In [117]:
print("Match rate")
pd.DataFrame({k1: {k2: results[(k1,k2)].shape[0]/addresses.shape[0] for k2 in ["struct", "unstruct"]} 
                  for k1 in  ["check","nocheck"]})

Match rate


,check,nocheck
struct,0.9780,0.9782
unstruct,0.9792,0.9799


In [116]:
print("Match rate (without nostreet)")
pd.DataFrame({k1: {k2: results[(k1,k2)].query("method!='nostreet'").shape[0]/addresses.shape[0] for k2 in ["struct", "unstruct"]} 
                  for k1 in  ["check","nocheck"]})

Match rate (without nostreet)


,check,nocheck
struct,0.9661,0.9669
unstruct,0.9666,0.9672


In [122]:
print("Unmatched addresses")
for k1 in results:
    print(k1)
    nomatch=addresses[~addresses[addr_key_field].isin(results[k1]["addr_key"])]
    display(nomatch)
    print(nomatch[country_field].value_counts())

Unmatched addresses
('check', 'struct')


,EntityNumber,CountryFR,Zipcode,MunicipalityFR,StreetFR,HouseNumber
1601520,0888.109.729,Pays-Bas,5462 GH,Veghel,De Amert,162
1563334,0881.292.312,Pays-Bas,3871 AN,Hoevelaken,drs. Van Royenstraat,5
443325,0506.848.655,Allemagne (Rép.féd.),98553,NAHETAL-WALDAU,SCHLEUSINGERSTRASSE,6
825163,0670.704.025,Suisse,1218,Le Grand-Saconnex,Route de Ferney,150
34669,0406.573.619,Congo (Rép. dém.),9999,Elisabethville,Avenue des Scieries,99
...,...,...,...,...,...,...
1200989,0817.157.494,Pays-Bas,5641JL,EINDHOVEN,Doolstraat,3
752841,0655.701.984,Suède,751 05,Uppsala,Uppsala Universitet,Box 256
1377168,0845.056.971,Pays-Bas,5343VD,OSS,Verdistraat,91
479556,0533.826.929,Pays-Bas,6215 VZ,MAASTRICHT,JUNOHOF,11


Pays-Bas                    63
France                      40
Allemagne (Rép.féd.)        24
Grande-Bretagne             20
Luxembourg (Grand-Duché)    18
Belgique                    12
Espagne                      4
Indéterminé                  4
Allemagne                    4
Italie                       4
Suisse                       4
Etats-Unis d'Amérique        3
Pologne ( Rép. )             2
Chine (Hong-Kong SAR)        2
Suède                        2
Portugal                     2
Pologne(République)          1
Irlande /Eire/               1
Iles Vierges                 1
Danemark                     1
Grèce                        1
Chypre                       1
République Slovaque          1
Sénégal                      1
Australie                    1
Congo (Rép. dém.)            1
Lituanie                     1
Emirats arabes unis          1
Name: CountryFR, dtype: int64
('check', 'unstruct')


,EntityNumber,CountryFR,Zipcode,MunicipalityFR,StreetFR,HouseNumber
1601520,0888.109.729,Pays-Bas,5462 GH,Veghel,De Amert,162
1563334,0881.292.312,Pays-Bas,3871 AN,Hoevelaken,drs. Van Royenstraat,5
443325,0506.848.655,Allemagne (Rép.féd.),98553,NAHETAL-WALDAU,SCHLEUSINGERSTRASSE,6
825163,0670.704.025,Suisse,1218,Le Grand-Saconnex,Route de Ferney,150
34669,0406.573.619,Congo (Rép. dém.),9999,Elisabethville,Avenue des Scieries,99
...,...,...,...,...,...,...
1200989,0817.157.494,Pays-Bas,5641JL,EINDHOVEN,Doolstraat,3
752841,0655.701.984,Suède,751 05,Uppsala,Uppsala Universitet,Box 256
1377168,0845.056.971,Pays-Bas,5343VD,OSS,Verdistraat,91
479556,0533.826.929,Pays-Bas,6215 VZ,MAASTRICHT,JUNOHOF,11


Pays-Bas                    62
France                      38
Allemagne (Rép.féd.)        24
Grande-Bretagne             20
Luxembourg (Grand-Duché)    18
Belgique                     5
Espagne                      4
Allemagne                    4
Italie                       4
Suisse                       4
Indéterminé                  3
Pologne ( Rép. )             2
Chine (Hong-Kong SAR)        2
Suède                        2
Portugal                     2
Etats-Unis d'Amérique        2
Iles Vierges                 1
Pologne(République)          1
Irlande /Eire/               1
Danemark                     1
Grèce                        1
Chypre                       1
Sénégal                      1
République Slovaque          1
Australie                    1
Congo (Rép. dém.)            1
Lituanie                     1
Emirats arabes unis          1
Name: CountryFR, dtype: int64
('nocheck', 'struct')


,EntityNumber,CountryFR,Zipcode,MunicipalityFR,StreetFR,HouseNumber
1601520,0888.109.729,Pays-Bas,5462 GH,Veghel,De Amert,162
1563334,0881.292.312,Pays-Bas,3871 AN,Hoevelaken,drs. Van Royenstraat,5
443325,0506.848.655,Allemagne (Rép.féd.),98553,NAHETAL-WALDAU,SCHLEUSINGERSTRASSE,6
825163,0670.704.025,Suisse,1218,Le Grand-Saconnex,Route de Ferney,150
34669,0406.573.619,Congo (Rép. dém.),9999,Elisabethville,Avenue des Scieries,99
...,...,...,...,...,...,...
1200989,0817.157.494,Pays-Bas,5641JL,EINDHOVEN,Doolstraat,3
752841,0655.701.984,Suède,751 05,Uppsala,Uppsala Universitet,Box 256
1377168,0845.056.971,Pays-Bas,5343VD,OSS,Verdistraat,91
479556,0533.826.929,Pays-Bas,6215 VZ,MAASTRICHT,JUNOHOF,11


Pays-Bas                    63
France                      40
Allemagne (Rép.féd.)        24
Grande-Bretagne             20
Luxembourg (Grand-Duché)    18
Belgique                    10
Espagne                      4
Indéterminé                  4
Allemagne                    4
Italie                       4
Suisse                       4
Etats-Unis d'Amérique        3
Pologne ( Rép. )             2
Chine (Hong-Kong SAR)        2
Suède                        2
Portugal                     2
Pologne(République)          1
Irlande /Eire/               1
Iles Vierges                 1
Danemark                     1
Grèce                        1
Chypre                       1
République Slovaque          1
Sénégal                      1
Australie                    1
Congo (Rép. dém.)            1
Lituanie                     1
Emirats arabes unis          1
Name: CountryFR, dtype: int64
('nocheck', 'unstruct')


,EntityNumber,CountryFR,Zipcode,MunicipalityFR,StreetFR,HouseNumber
1601520,0888.109.729,Pays-Bas,5462 GH,Veghel,De Amert,162
1563334,0881.292.312,Pays-Bas,3871 AN,Hoevelaken,drs. Van Royenstraat,5
443325,0506.848.655,Allemagne (Rép.féd.),98553,NAHETAL-WALDAU,SCHLEUSINGERSTRASSE,6
825163,0670.704.025,Suisse,1218,Le Grand-Saconnex,Route de Ferney,150
34669,0406.573.619,Congo (Rép. dém.),9999,Elisabethville,Avenue des Scieries,99
...,...,...,...,...,...,...
1200989,0817.157.494,Pays-Bas,5641JL,EINDHOVEN,Doolstraat,3
752841,0655.701.984,Suède,751 05,Uppsala,Uppsala Universitet,Box 256
1377168,0845.056.971,Pays-Bas,5343VD,OSS,Verdistraat,91
479556,0533.826.929,Pays-Bas,6215 VZ,MAASTRICHT,JUNOHOF,11


Pays-Bas                    62
France                      37
Allemagne (Rép.féd.)        24
Grande-Bretagne             20
Luxembourg (Grand-Duché)    12
Belgique                     5
Espagne                      4
Allemagne                    4
Italie                       4
Suisse                       4
Indéterminé                  3
Pologne ( Rép. )             2
Chine (Hong-Kong SAR)        2
Suède                        2
Portugal                     2
Etats-Unis d'Amérique        2
Iles Vierges                 1
Pologne(République)          1
Irlande /Eire/               1
Danemark                     1
Grèce                        1
Chypre                       1
Sénégal                      1
République Slovaque          1
Australie                    1
Congo (Rép. dém.)            1
Lituanie                     1
Emirats arabes unis          1
Name: CountryFR, dtype: int64


In [123]:
vc_values = pd.DataFrame(columns=results.keys(), index=results.keys())

for k1 in results:
    vc_values.loc[k1, k1] = results[k1].shape[0]
    for k2 in results:
        if k1>k2:
            r1=results[k1]
            r2=results[k2]
            mg = r1[["addr_key", "place_id"]].merge(r2[["addr_key", "place_id"]], on="addr_key", how="outer", indicator=True)
 
            vc = mg._merge.value_counts()

            mismatches = mg[mg.place_id_x != mg.place_id_y][["addr_key"]]
            mismatches = mismatches.merge(addresses.rename({addr_key_field:"addr_key"}, axis=1))
            mismatches = mismatches.merge(r1[["addr_key", "addr_out_street", "addr_out_number", "extra_house_nbr", "addr_out_postcode", "addr_out_city"]], on="addr_key")
            mismatches = mismatches.merge(r2[["addr_key", "addr_out_street", "addr_out_number", "extra_house_nbr", "addr_out_postcode", "addr_out_city"]], on="addr_key")
            mismatches.columns = pd.MultiIndex.from_arrays([["Input"]*6 + [f"x:{k1}"]*5 + [f"y:{k2}"]*5, mismatches.columns])

            mismatch_values = mismatches[(mismatches[f"x:{k1}"].rename(lambda x: x.replace("_x", ""), axis=1).fillna("") != 
                                          mismatches[f"y:{k2}"].rename(lambda x: x.replace("_y", ""), axis=1).fillna("")).any(axis=1)]
            
            mismatch_values_no_nmbr = mismatches[(mismatches[f"x:{k1}"].rename(lambda x: x.replace("_x", ""), axis=1).drop("addr_out_number", axis=1).fillna("") != 
                                                  mismatches[f"y:{k2}"].rename(lambda x: x.replace("_y", ""), axis=1).drop("addr_out_number", axis=1).fillna("")).any(axis=1)]
            
            
            vc_label = f"{vc['both']} ({mismatches.shape[0]} - {mismatch_values.shape[0]} - {mismatch_values_no_nmbr.shape[0]}) / {vc['left_only']} / {vc['right_only']}"
            vc_values.loc[k1, k2]=vc_label

                
            print(f"{k1} vs {k2}")
            print(vc_label)
            print("-----------------------------")
            
            print(f"Only in {k1}")
            display(r1[r1.addr_key.isin(mg[mg._merge=="left_only"].addr_key)].merge(addresses.rename({addr_key_field:"addr_key"}, axis=1)))
            
            print(f"Only in {k2}")
            display(r2[r2.addr_key.isin(mg[mg._merge=="right_only"].addr_key)].merge(addresses.rename({addr_key_field:"addr_key"}, axis=1)))
            
            print("Mismatch on place_id")
            display(mismatches)
            
            print("Mismatch on values")
            
            display(mismatch_values)
            
            print("Mismatch on values (no nbr)")
            display(mismatch_values_no_nmbr)
            
            print("#######################")
            
# display(vc_values.fillna(""))

('check', 'unstruct') vs ('check', 'struct')
9779 (332 - 297 - 225) / 13 / 1
-----------------------------
Only in ('check', 'unstruct')


,addr_key,lat,lon,place_rank,method,place_id,addr_out_street,addr_out_number,extra_house_nbr,addr_out_postcode,addr_out_city,addr_out_country,CountryFR,Zipcode,MunicipalityFR,StreetFR,HouseNumber
0,0718.571.347,50.092422,5.281180,26.0,orig,792752.0,Rue de la Victoire,None,10,6927,Grupont,België / Belgique / Belgien,Belgique,6927,Tellin,"Rue de la Victoire,Grupont",10
1,0415.302.332,50.736920,3.194214,26.0,libpostal+regex[lpost]+photon,635571.0,Rue Ampère,None,22,None,None,France,France,nan,Grenoble,Rue Ampere,22
2,0542.730.737,50.573082,3.896295,21.0,nostreet,4068548.0,None,None,1,7870,Mons,België / Belgique / Belgien,Belgique,7870,Lens (Hainaut),Rue Beaufosse(L),1
3,2.027.168.257,50.659730,5.540637,30.0,libpostal+regex[lpost],4004314.0,Rue des Trois Rois,111,111,4430,Ans,België / Belgique / Belgien,Indéterminé,4000,ANS (LIEGE),111 R DES TROIS ROIS,NaN
4,2.143.761.465,50.473772,4.450385,21.0,nostreet,4068263.0,None,None,5,6041,Gosselies,België / Belgique / Belgien,Belgique,6041,Charleroi,Rue Santos - Dumont,5
5,2.065.887.192,50.167362,4.888208,30.0,orig,2963997.0,Rue d'Inzepia,8,8,5560,Mesnil-Saint-Blaise,België / Belgique / Belgien,Belgique,5560,Houyet,"Rue d'Inzepia,M.-St-Blaise",8
6,2.198.913.784,50.087950,5.839892,26.0,orig,1148156.0,Boeur,None,14,None,Boeur,België / Belgique / Belgien,Belgique,6662,Houffalize,Boeur,14
7,0841.745.412,50.816434,4.327111,21.0,nostreet,4067596.0,None,None,67,1190,Forest - Vorst,België / Belgique / Belgien,Belgique,1190,Forest,Bd 2me Armée Britannique,67
8,0416.114.063,50.365059,5.808542,26.0,nonum,1645383.0,Haute Bodeux,None,31,4980,Haute-Bodeux,België / Belgique / Belgien,Belgique,4983,Trois-Ponts,Haute-Bodeux,31
9,0465.353.540,51.282600,5.306129,16.0,nostreet,4047887.0,None,None,23,5575XD,Luyksgestel,Nederland,Pays-Bas,nan,Luyksgestel,Hoge Rijt,23


Only in ('check', 'struct')


,addr_key,lat,lon,place_rank,method,place_id,addr_out_street,addr_out_number,extra_house_nbr,addr_out_postcode,addr_out_city,addr_out_country,CountryFR,Zipcode,MunicipalityFR,StreetFR,HouseNumber
0,0410.438.870,51.076635,4.128812,26.0,libpostal+regex[lpost]+photon,618729.0,Kraneplas,None,2,9220,Hamme,België / Belgique / Belgien,Belgique,9220,Hamme (Vl.),Kraneplas,2


Mismatch on place_id


...

Mismatch on values


Input                                           \
          addr_key CountryFR Zipcode         MunicipalityFR   
0     0722.431.155  Belgique    5555                 Bièvre   
1     0681.048.579  Belgique    8920  Langemark-Poelkapelle   
2     0840.713.252  Belgique    9600                  Ronse   
3    2.221.412.044  Belgique    1050                Ixelles   
4    2.249.457.120  Belgique    2018              Antwerpen   
..             ...       ...     ...                    ...   
346   0730.428.509  Belgique    3740                 Bilzen   
347  2.253.839.342  Belgique    6730               Tintigny   
348  2.248.770.596  Belgique    6972             Tenneville   
349  2.226.148.713  Belgique    5070        Fosses-la-Ville   
351   0540.940.690  Belgique    8700                  Tielt   

                                                  x:('nocheck', 'unstruct')  \
                           StreetFR HouseNumber           addr_out_street_x   
0                 Village, M.-en-A.          10                     Village   
1                         Brugseweg         208                   Brugseweg   
2    Rue des Prisonniers-Politiques           3                        None   
3                     Avenue Louise         379  Avenue Louise - Louizalaan   
4                Lange Lozanastraat       246/2          Lange Lozanastraat   
..                              ...         ...                         ...   
346               Kleehofkensstraat          62                        None   
347        Rue du Centenaire,Ansart          44           Rue du Centenaire   
348                     Grand'Garde          11                 Grand'Garde   
349      Rue de la Bruyère,Vitrival          24           Rue de la Bruyère   
351                  Tereikenstraat           3                        None   

                                                             \
    addr_out_number_x extra_house_nbr_x addr_out_postcode_x   
0                None                10                5555   
1                None               208                8920   
2                None                 3                9600   
3                 379               379                1000   
4             246-248             246/2                2018   
..                ...               ...                 ...   
346              None                62                3740   
347                44                44                6730   
348              None                11                6970   
349                24                24                5070   
351              None                 3                8700   

                                                    y:('nocheck', 'struct')  \
         addr_out_city_x                                  addr_out_street_y   
0          Bellefontaine                                      Rue de Bièvre   
1              Langemark                                          Brugseweg   
2             Oudenaarde                             Winston Churchillplein   
3    Bruxelles / Brussel        Avenue de la Toison d'Or - Gulden-Vlieslaan   
4              Antwerpen                                 Lange Lozanastraat   
..                   ...                                                ...   
346               Bilzen                                         Zeepstraat   
347             Tintigny                                  Rue du Centenaire   
348        Trèsfontaines                                        Grand'Garde   
349      Fosses-la-Ville  Rue de La Bruyère (Chemin vicinal n°4 de Vitri...   
351                Tielt                                       Stationsberm   

                                                                               
    addr_out_number_y extra_house_nbr_y addr_out_postcode_y   addr_out_city_y  
0                None                10                5555        Carlsbourg  
1                None               208                8920       Poelkapelle

Mismatch on values (no nbr)


Input                                           \
          addr_key CountryFR Zipcode         MunicipalityFR   
0     0722.431.155  Belgique    5555                 Bièvre   
1     0681.048.579  Belgique    8920  Langemark-Poelkapelle   
2     0840.713.252  Belgique    9600                  Ronse   
3    2.221.412.044  Belgique    1050                Ixelles   
5     0688.311.406  Belgique    5590                  Ciney   
..             ...       ...     ...                    ...   
344   0886.858.627  Belgique    7070              Le Roeulx   
345  2.228.026.355  Belgique    1050                Ixelles   
346   0730.428.509  Belgique    3740                 Bilzen   
349  2.226.148.713  Belgique    5070        Fosses-la-Ville   
351   0540.940.690  Belgique    8700                  Tielt   

                                                  x:('nocheck', 'unstruct')  \
                           StreetFR HouseNumber           addr_out_street_x   
0                 Village, M.-en-A.          10                     Village   
1                         Brugseweg         208                   Brugseweg   
2    Rue des Prisonniers-Politiques           3                        None   
3                     Avenue Louise         379  Avenue Louise - Louizalaan   
5             Rue de Mont(Braibant)          40                 Rue de Mont   
..                              ...         ...                         ...   
344         Chaussée de Soignies(R)          22        Chaussée de Soignies   
345                   Avenue Louise         523  Avenue Louise - Louizalaan   
346               Kleehofkensstraat          62                        None   
349      Rue de la Bruyère,Vitrival          24           Rue de la Bruyère   
351                  Tereikenstraat           3                        None   

                                                             \
    addr_out_number_x extra_house_nbr_x addr_out_postcode_x   
0                None                10                5555   
1                None               208                8920   
2                None                 3                9600   
3                 379               379                1000   
5                  40                40                5590   
..                ...               ...                 ...   
344              None                22                7070   
345               523               523                1000   
346              None                62                3740   
349                24                24                5070   
351              None                 3                8700   

                                                    y:('nocheck', 'struct')  \
         addr_out_city_x                                  addr_out_street_y   
0          Bellefontaine                                      Rue de Bièvre   
1              Langemark                                          Brugseweg   
2             Oudenaarde                             Winston Churchillplein   
3    Bruxelles / Brussel        Avenue de la Toison d'Or - Gulden-Vlieslaan   
5                  Ciney                                    Rue de Braibant   
..                   ...                                                ...   
344             Soignies          Chaussée du Roeulx - Chaussée de Soignies   
345  Bruxelles / Brussel        Avenue de la Toison d'Or - Gulden-Vlieslaan   
346               Bilzen                                         Zeepstraat   
349      Fosses-la-Ville  Rue de La Bruyère (Chemin vicinal n°4 de Vitri...   
351                Tielt                                       Stationsberm   

                                                                               
    addr_out_number_y extra_house_nbr_y addr_out_postcode_y   addr_out_city_y  
0                None                10                5555        Carlsbourg  
1                None               208                8920       Poelkapelle

#######################


In [124]:
print("Common in both (disagree on place_id - disagree on values - disagree on values, ignoring number) / results only for row / results only for columns")
vc_values.fillna("")

Common in both (disagree on place_id - disagree on values - disagree on values, ignoring number) / results only for row / results only for columns


check  \
                                           struct   
check   struct                               9780   
        unstruct  9779 (332 - 297 - 225) / 13 / 1   
nocheck struct     9780 (255 - 255 - 252) / 2 / 0   
        unstruct  9779 (506 - 471 - 397) / 20 / 1   

                                                   \
                                         unstruct   
check   struct                                      
        unstruct                             9792   
nocheck struct    9781 (521 - 483 - 410) / 1 / 11   
        unstruct   9792 (192 - 192 - 189) / 7 / 0   

                                          nocheck           
                                           struct unstruct  
check   struct                                              
        unstruct                                            
nocheck struct                               9782           
        unstruct  9781 (352 - 313 - 244) / 18 / 1     9799